In [1]:
import flwr as fl
print(f"Flower version: {fl.__version__}")


Flower version: 1.13.0


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import shuffle

# Dataset names
dataset_names = [
    "test.csv", "train.csv", "all.csv", "dev.csv", "test_hidden.csv",
    "train_fewshot-128.csv", "train_fewshot-32.csv",
    "train_fewshot-512.csv", "train_fewshot-8.csv"
]

# Load datasets into a dictionary
datasets = {}
for dataset_name in dataset_names:
    try:
        df = pd.read_csv(dataset_name)
        datasets[dataset_name] = df
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")

# Preprocess data - ensuring the 'ad_creative_body' column is present
for name, df in datasets.items():
    if 'ad_creative_body' not in df.columns:
        print(f"Warning: 'ad_creative_body' column not found in {name}")
    else:
        print(f"Loaded {name} with {len(df)} rows.")

# Preprocess text data (cleaning and normalization)
def preprocess_text(text):
    # Remove special characters, convert to lowercase, and return
    text = text.lower()
    text = ''.join(e for e in text if e.isalnum() or e.isspace())
    return text

# Prepare the vectorizer and labels
all_text = []
for df in datasets.values():
    if not df.empty and 'ad_creative_body' in df.columns:
        all_text.extend(df["ad_creative_body"].apply(preprocess_text).values)

# Create a global TF-IDF vectorizer
global_vectorizer = TfidfVectorizer(stop_words='english')
global_vectorizer.fit(all_text)

# Federated Client class
class FederatedClient:
    def __init__(self, name, X_train, X_test, y_train, y_test):
        self.name = name
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.model = self.build_model()

    def build_model(self):
        # Build a Neural Network Model
        model = Sequential()
        model.add(Dense(512, input_dim=self.X_train.shape[1], activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(max_classes, activation='softmax'))  # For multi-class classification
        
        model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    def train(self):
        self.model.fit(self.X_train, self.y_train, epochs=5, batch_size=32, verbose=2)

    def evaluate(self):
        y_pred = np.argmax(self.model.predict(self.X_test), axis=1)
        return accuracy_score(self.y_test, y_pred), classification_report(self.y_test, y_pred), confusion_matrix(self.y_test, y_pred)

    def get_parameters(self):
        return self.model.get_weights()

    def set_parameters(self, parameters):
        self.model.set_weights(parameters)

# Prepare Federated Learning clients
clients = []
max_classes = 0  # Track the maximum number of classes across clients
for name, df in datasets.items():
    if 'ad_creative_body' in df.columns and 'Typology 1' in df.columns:
        X = df["ad_creative_body"].apply(preprocess_text).values
        y = df["Typology 1"].values.astype(str)  # Convert target labels to string

        valid_rows = ~pd.isna(y)
        X = X[valid_rows]
        y = y[valid_rows]

        if len(np.unique(y)) < 2:
            print(f"Skipping {name} as it contains fewer than 2 classes.")
            continue

        max_classes = max(max_classes, len(np.unique(y)))

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train_tfidf = global_vectorizer.transform(X_train).toarray()
        X_test_tfidf = global_vectorizer.transform(X_test).toarray()

        # Encode the labels as integers
        label_map = {label: idx for idx, label in enumerate(np.unique(y))}
        y_train_encoded = np.array([label_map[label] for label in y_train])
        y_test_encoded = np.array([label_map[label] for label in y_test])

        client = FederatedClient(name, X_train_tfidf, X_test_tfidf, y_train_encoded, y_test_encoded)
        clients.append(client)

# Federated Training function
def federated_training(clients, num_rounds=10):
    global_weights = clients[0].model.get_weights()

    for round_num in range(num_rounds):
        local_weights = []

        # Train each client and collect their weights
        for client in clients:
            client.set_parameters(global_weights)
            client.train()
            local_weights.append(client.get_parameters())

        # Aggregate weights (FedAvg)
        avg_weights = [np.mean(np.array([local_weights[i][j] for i in range(len(local_weights))]), axis=0) for j in range(len(local_weights[0]))]
        global_weights = avg_weights

        # Update clients with the new global weights
        for client in clients:
            client.set_parameters(global_weights)

        # Evaluate the clients
        accuracies = []
        for client in clients:
            try:
                accuracy, report, conf_matrix = client.evaluate()
                print(f"Client {client.name} - Accuracy: {accuracy:.4f}")
                accuracies.append(accuracy)
                print("Classification Report:")
                print(report)
                print("Confusion Matrix:")
                print(conf_matrix)
            except IndexError as e:
                print(f"Evaluation error for client {client.name}: {e}")
                accuracies.append(0)  # Assign zero accuracy if evaluation fails

        avg_accuracy = np.mean(accuracies)
        print(f"Round {round_num + 1}/{num_rounds}, Average accuracy: {avg_accuracy:.4f}")

 
#  For Running  federated training and  prediction                              
federated_training(clients, num_rounds=10)


Loaded test.csv with 255 rows.
Loaded train.csv with 913 rows.
Loaded all.csv with 1330 rows.
Loaded dev.csv with 162 rows.
Loaded test_hidden.csv with 255 rows.
Loaded train_fewshot-128.csv with 128 rows.
Loaded train_fewshot-32.csv with 32 rows.
Loaded train_fewshot-512.csv with 512 rows.
Loaded train_fewshot-8.csv with 8 rows.


C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Skipping test_hidden.csv as it contains fewer than 2 classes.
Epoch 1/5
7/7 - 6s - 822ms/step - accuracy: 0.2598 - loss: 2.0493
Epoch 2/5
7/7 - 0s - 48ms/step - accuracy: 0.3235 - loss: 1.8890
Epoch 3/5
7/7 - 0s - 53ms/step - accuracy: 0.3333 - loss: 1.6324
Epoch 4/5
7/7 - 0s - 47ms/step - accuracy: 0.4657 - loss: 1.3868
Epoch 5/5
7/7 - 0s - 46ms/step - accuracy: 0.6667 - loss: 1.0758
Epoch 1/5
23/23 - 6s - 266ms/step - accuracy: 0.2575 - loss: 1.9384
Epoch 2/5
23/23 - 1s - 35ms/step - accuracy: 0.5438 - loss: 1.4379
Epoch 3/5
23/23 - 1s - 36ms/step - accuracy: 0.7521 - loss: 0.7443
Epoch 4/5
23/23 - 1s - 36ms/step - accuracy: 0.8712 - loss: 0.3764
Epoch 5/5
23/23 - 1s - 35ms/step - accuracy: 0.9644 - loss: 0.1785
Epoch 1/5
34/34 - 6s - 185ms/step - accuracy: 0.3148 - loss: 1.8645
Epoch 2/5
34/34 - 1s - 34ms/step - accuracy: 0.6212 - loss: 1.1705
Epoch 3/5
34/34 - 1s - 40ms/step - accuracy: 0.8289 - loss: 0.4989
Epoch 4/5
34/34 - 1s - 34ms/step - accuracy: 0.9492 - loss: 0.2216
Epoch 5

C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/6 ━━━━━━━━━━━━━━━━━━━━ 1s 229ms/stepWARNING:tensorflow:6 out of the last 31 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000204D872B7E0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Client train.csv - Accuracy: 0.6885
Classification Report:
              precision    recall  f1-score   support

           0       0.48      1.00      0.65        32
           1       0.00      

...

C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\

7/7 - 0s - 43ms/step - accuracy: 1.0000 - loss: 0.0055
Epoch 2/5
7/7 - 0s - 44ms/step - accuracy: 0.9951 - loss: 0.0076
Epoch 3/5
7/7 - 0s - 44ms/step - accuracy: 1.0000 - loss: 0.0029
Epoch 4/5
7/7 - 0s - 43ms/step - accuracy: 1.0000 - loss: 0.0020
Epoch 5/5
7/7 - 0s - 44ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 1/5
23/23 - 1s - 34ms/step - accuracy: 1.0000 - loss: 0.0023
Epoch 2/5
23/23 - 1s - 36ms/step - accuracy: 1.0000 - loss: 9.9361e-04
Epoch 3/5
23/23 - 1s - 59ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 4/5
23/23 - 1s - 35ms/step - accuracy: 1.0000 - loss: 7.1116e-04
Epoch 5/5
23/23 - 1s - 35ms/step - accuracy: 1.0000 - loss: 5.2031e-04
Epoch 1/5
34/34 - 1s - 33ms/step - accuracy: 0.9972 - loss: 0.0098
Epoch 2/5
34/34 - 1s - 33ms/step - accuracy: 1.0000 - loss: 0.0020
Epoch 3/5
34/34 - 1s - 33ms/step - accuracy: 1.0000 - loss: 9.1026e-04
Epoch 4/5
34/34 - 1s - 34ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 5/5
34/34 - 1s - 40ms/step - accuracy: 1.0000 - loss: 0.0012

C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Client train_fewshot-128.csv - Accuracy: 0.9615
Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         2
           6       1.00      0.50      0.67         2
           7       1.00      1.00      1.00         4

    accuracy                           0.96        26
   macro avg       0.98      0.93      0.94        26
weighted avg       0.97      0.96      0.96        26

Confusion Matrix:
[[6 0 0 0 0 0 0]
 [0 2 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 6 0 0 0]
 [0 0 0 0 2 0 0]
 [1 0 0 0 0 1 0]
 [0 0 0 0 0 0 4]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Client train_fewshot-32.csv - Accuracy: 1.0000
Classification Report:
              precision    recall  f1-score   su

C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\OneDrive\Desktop\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\krish\